In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import gc
import os
import matplotlib
import matplotlib.pyplot as plt


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pd.set_option('max_columns', 150)

# matplotlib and seaborn for plotting

matplotlib.rcParams['figure.dpi'] = 120 #resolution
matplotlib.rcParams['figure.figsize'] = (8,6) #figure size


sns.set_style('darkgrid')

color = sns.color_palette()
rootdata = '/content/drive/MyDrive/Thesis Data/original_data/'
destination_data = '/content/drive/MyDrive/Thesis Data/prepared_data/'

# Data Loading

In [4]:
aisles = pd.read_csv(rootdata+'aisles.csv')
departments = pd.read_csv(rootdata+'departments.csv')
orders = pd.read_csv(rootdata+'orders.csv')
order_products_prior = pd.read_csv(rootdata+'order_products__prior.csv')
order_products_train = pd.read_csv(rootdata+'order_products__train.csv')
products = pd.read_csv(rootdata+'products.csv')

In [5]:
# Method for memory reduction 
def reduce_memory(df):
   
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    
    for col in df.columns:
        if df[col].dtypes in ["int64", "int32", "int16"]:
            
            cmin = df[col].min()
            cmax = df[col].max()
            
            if cmin > np.iinfo(np.int8).min and cmax < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            
            elif cmin > np.iinfo(np.int16).min and cmax < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            
            elif cmin > np.iinfo(np.int32).min and cmax < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
        
        if df[col].dtypes in ["float64", "float32"]:
            
            cmin = df[col].min()
            cmax = df[col].max()
            
            if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            
            elif cmin > np.finfo(np.float32).min and cmax < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
    
    print("")
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    
    return df

#Data Cleaning

In [6]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


#### Null value imputation  for days_since_prior_order

In [7]:
# verifying which all feature is having null values.
round(orders.isnull().sum(axis=0)/(orders.shape[0])*100).sort_values(ascending=False)

days_since_prior_order    6.0
order_hour_of_day         0.0
order_dow                 0.0
order_number              0.0
eval_set                  0.0
user_id                   0.0
order_id                  0.0
dtype: float64

In [8]:
orders.columns [orders.isna().any()]

Index(['days_since_prior_order'], dtype='object')

In [9]:
orders['days_since_prior_order'].value_counts()

30.0    369323
7.0     320608
6.0     240013
4.0     221696
3.0     217005
5.0     214503
2.0     193206
8.0     181717
1.0     145247
9.0     118188
14.0    100230
10.0     95186
13.0     83214
11.0     80970
12.0     76146
0.0      67755
15.0     66579
16.0     46941
21.0     45470
17.0     39245
20.0     38527
18.0     35881
19.0     34384
22.0     32012
28.0     26777
23.0     23885
27.0     22013
24.0     20712
25.0     19234
29.0     19191
26.0     19016
Name: days_since_prior_order, dtype: int64

In [10]:
#Impute null value with median
orders['days_since_prior_order'].fillna(orders.days_since_prior_order.median(),inplace=True)

In [11]:
orders.columns [orders.isna().any()]

Index([], dtype='object')

# Data Transformation & Feature Selection 

In [12]:
# Merging three dataset together (orders vs order_products_prior vs products) to get product purchase  history  details
product_purchase_history_df = order_products_prior.merge(orders, on ='order_id', how='inner')
product_purchase_history_df = product_purchase_history_df.merge(products, on = 'product_id', how = 'left')
product_purchase_history_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13


In [13]:
#Memory reduction  upto 41% from initial dataset
product_purchase_history_df = reduce_memory(product_purchase_history_df)

Memory usage of properties dataframe is : 3464.377182006836  MB

___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  1453.8011388778687  MB
This is  41.964285714285715 % of the initial size


###### Data transformation : user purchase product count

In [14]:
#Feature : user_purchased_product_count
product_purchase_history_df['user_purchased_product_count'] = product_purchase_history_df.groupby(['user_id', 'product_id']).cumcount() + 1


In [15]:
product_purchase_history_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_purchased_product_count
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


##### Data transformation : Product level feature selection

In [16]:
product_level_features = pd.DataFrame(columns=['product_avg_add_to_cart_order'])

In [17]:
#Feature : product_avg_add_to_cart_order
product_level_features['product_avg_add_to_cart_order'] = product_purchase_history_df.groupby('product_id')['add_to_cart_order'].mean()

In [18]:
#Feature : product_orders_total_count
product_level_features['product_orders_total_count'] = product_purchase_history_df.groupby('product_id')['reordered'].count()

In [19]:
#Feature : product_reorder_total_count
product_level_features['product_reorder_total_count'] = product_purchase_history_df.groupby('product_id')['reordered'].sum()

In [20]:
#Feature : prod_order_first_time_total_cnt
product_level_features['product_reorder_percentage'] = product_purchase_history_df.groupby('product_id')['reordered'].mean()

In [21]:
#Feature : prod_order_first_time_total_cnt
product_level_features['prod_order_first_time_total_cnt'] = product_purchase_history_df.groupby('product_id')['user_purchased_product_count'].apply(lambda x: sum(x==1))

In [22]:
#Feature : prod_order_second_time_total_cnt
product_level_features['prod_order_second_time_total_cnt'] = product_purchase_history_df.groupby('product_id')['user_purchased_product_count'].apply(lambda x: sum(x==2))

In [23]:
#Feature : product_is_organic
product_level_features['product_is_organic'] = product_purchase_history_df.groupby('product_id')['product_name'].apply(lambda x:1 if 'Organic' in x else 0)

In [24]:
#Feature : product_vs_unique_users
product_level_features['product_vs_unique_users' ] = product_purchase_history_df.groupby('product_id')['user_id'].apply(lambda x: x.nunique())

In [25]:
#Feature : second_time_purchased_percent
product_level_features['second_time_purchased_percent'] = product_level_features.prod_order_second_time_total_cnt/product_level_features.prod_order_first_time_total_cnt

In [26]:
product_level_features.head()

,product_avg_add_to_cart_order,product_orders_total_count,product_reorder_total_count,product_reorder_percentage,prod_order_first_time_total_cnt,prod_order_second_time_total_cnt,product_is_organic,product_vs_unique_users,second_time_purchased_percent
product_id,,,,,,,,,
1,5.801836,1852,1136.0,0.613391,716,276,0,716,0.385475
2,9.888889,90,12.0,0.133333,78,8,0,78,0.102564
3,6.415162,277,203.0,0.732852,74,36,0,74,0.486486
4,9.507599,329,147.0,0.446809,182,64,0,182,0.351648
5,6.466667,15,9.0,0.600000,6,4,0,6,0.666667


##### Data transformation : Aisle level feature selection

In [27]:
aisle_level_features = pd.DataFrame(columns=['aisle_avg_add_to_cart_order'])

In [28]:
#Feature : aisle_avg_add_to_cart_order
aisle_level_features['aisle_avg_add_to_cart_order'] = product_purchase_history_df.groupby('aisle_id')['add_to_cart_order'].mean()

In [29]:
#Feature : aisle_std_add_to_cart_order
aisle_level_features['aisle_std_add_to_cart_order'] = product_purchase_history_df.groupby('aisle_id')['add_to_cart_order'].std()

In [30]:
#Feature : aisle_orders_total_count
aisle_level_features['aisle_orders_total_count'] = product_purchase_history_df.groupby('aisle_id')['reordered'].count()

In [31]:
#Feature : aisle_reorder_total_count
aisle_level_features['aisle_reorder_total_count'] = product_purchase_history_df.groupby('aisle_id')['reordered'].sum()

In [32]:
#Feature : aisle_reorder_percentage
aisle_level_features['aisle_reorder_percentage'] = product_purchase_history_df.groupby('aisle_id')['reordered'].mean()

In [33]:
#Feature : aisle_vs_unique_users
aisle_level_features['aisle_vs_unique_users'] = product_purchase_history_df.groupby('aisle_id')['user_id'].apply(lambda x: x.nunique())

In [34]:
aisle_level_features.head()

,aisle_avg_add_to_cart_order,aisle_std_add_to_cart_order,aisle_orders_total_count,aisle_reorder_total_count,aisle_reorder_percentage,aisle_vs_unique_users
aisle_id,,,,,,
1,8.167640,7.104166,71928,42912.0,0.596597,20711
2,9.275497,7.473802,82491,40365.0,0.489326,31222
3,9.571935,7.899672,456386,272922.0,0.598007,63592
4,10.161450,7.745705,200687,98243.0,0.489533,53892
5,10.297600,8.187047,62510,17542.0,0.280627,32312


##### Data transformation : Department level feature selection

In [35]:
department_level_features = pd.DataFrame(columns=['department_avg_add_to_cart_order'])

In [36]:
#Feature : department_avg_add_to_cart_order
department_level_features['department_avg_add_to_cart_order'] = product_purchase_history_df.groupby('department_id')['add_to_cart_order'].mean()

In [37]:
#Feature : department_std_add_to_cart_order
department_level_features['department_std_add_to_cart_order'] = product_purchase_history_df.groupby('department_id')['add_to_cart_order'].std()

In [38]:
#Feature : department_orders_total_count
department_level_features['department_orders_total_count'] = product_purchase_history_df.groupby('department_id')['reordered'].count()

In [39]:
#Feature : department_reorder_total_count
department_level_features['department_reorder_total_count'] = product_purchase_history_df.groupby('department_id')['reordered'].sum()

In [40]:
#Feature : department_reorder_percentage
department_level_features['department_reorder_percentage'] = product_purchase_history_df.groupby('department_id')['reordered'].mean()

In [41]:
#Feature : department_vs_unique_users
department_level_features['department_vs_unique_users'] = product_purchase_history_df.groupby('department_id')['user_id'].apply(lambda x: x.nunique())

In [42]:
department_level_features.head()

,department_avg_add_to_cart_order,department_std_add_to_cart_order,department_orders_total_count,department_reorder_total_count,department_reorder_percentage,department_vs_unique_users
department_id,,,,,,
1,8.996414,7.393502,2236432,1211890.0,0.541885,163233
2,8.277645,7.526272,36291,14806.0,0.407980,17875
3,8.084397,6.904849,1176787,739188.0,0.628141,140612
4,8.022875,6.658899,9479291,6160710.0,0.649913,193237
5,5.428346,5.778253,153696,87595.0,0.569924,15798


####  Merging five dataset together for getting relational dataset with products vs departments  vs aisles and storing it to product_level feature

In [43]:
product_level_features = product_level_features.merge(products, on = 'product_id', how = 'left')
product_level_features = product_level_features.merge(aisle_level_features, on = 'aisle_id', how = 'left')
product_level_features = product_level_features.merge(aisles, on = 'aisle_id', how = 'left')
product_level_features = product_level_features.merge(department_level_features, on = 'department_id', how = 'left')
product_level_features = product_level_features.merge(departments, on = 'department_id', how = 'left')
product_level_features.head()

,product_id,product_avg_add_to_cart_order,product_orders_total_count,product_reorder_total_count,product_reorder_percentage,prod_order_first_time_total_cnt,prod_order_second_time_total_cnt,product_is_organic,product_vs_unique_users,second_time_purchased_percent,product_name,aisle_id,department_id,aisle_avg_add_to_cart_order,aisle_std_add_to_cart_order,aisle_orders_total_count,aisle_reorder_total_count,aisle_reorder_percentage,aisle_vs_unique_users,aisle,department_avg_add_to_cart_order,department_std_add_to_cart_order,department_orders_total_count,department_reorder_total_count,department_reorder_percentage,department_vs_unique_users,department
0,1,5.801836,1852,1136.0,0.613391,716,276,0,716,0.385475,Chocolate Sandwich Cookies,61,19,9.253092,7.845107,234065,128431.0,0.548698,54202,cookies cakes,9.187743,7.692492,2887550,1657973.0,0.574180,174219,snacks
1,2,9.888889,90,12.0,0.133333,78,8,0,78,0.102564,All-Seasons Salt,104,13,9.996181,7.994790,212092,32321.0,0.152391,76402,spices seasonings,9.593425,7.875241,1875577,650301.0,0.346721,172755,pantry
2,3,6.415162,277,203.0,0.732852,74,36,0,74,0.486486,Robust Golden Unsweetened Oolong Tea,94,7,8.519846,7.818249,249341,131556.0,0.527615,53197,tea,6.976699,6.711172,2690129,1757892.0,0.653460,172795,beverages
3,4,9.507599,329,147.0,0.446809,182,64,0,182,0.351648,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,9.207741,7.435740,390299,217262.0,0.556655,58749,frozen meals,8.996414,7.393502,2236432,1211890.0,0.541885,163233,frozen
4,5,6.466667,15,9.0,0.600000,6,4,0,6,0.666667,Green Chile Anytime Sauce,5,13,10.297600,8.187047,62510,17542.0,0.280627,32312,marinades meat preparation,9.593425,7.875241,1875577,650301.0,0.346721,172755,pantry


In [44]:
product_level_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49677 entries, 0 to 49676
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   product_id                        49677 non-null  int64  
 1   product_avg_add_to_cart_order     49677 non-null  float64
 2   product_orders_total_count        49677 non-null  int64  
 3   product_reorder_total_count       49677 non-null  float64
 4   product_reorder_percentage        49677 non-null  float64
 5   prod_order_first_time_total_cnt   49677 non-null  int64  
 6   prod_order_second_time_total_cnt  49677 non-null  int64  
 7   product_is_organic                49677 non-null  int64  
 8   product_vs_unique_users           49677 non-null  int64  
 9   second_time_purchased_percent     49677 non-null  float64
 10  product_name                      49677 non-null  object 
 11  aisle_id                          49677 non-null  int64  
 12  depa

#### Elimination of Variable 

In [45]:
#product_name , aisle_id and department _id will not play any. role for model building . Hence eliminating  from the dataset
product_level_features.drop(['product_name', 'aisle_id', 'department_id'], axis = 1, inplace = True)
product_level_features.head()

,product_id,product_avg_add_to_cart_order,product_orders_total_count,product_reorder_total_count,product_reorder_percentage,prod_order_first_time_total_cnt,prod_order_second_time_total_cnt,product_is_organic,product_vs_unique_users,second_time_purchased_percent,aisle_avg_add_to_cart_order,aisle_std_add_to_cart_order,aisle_orders_total_count,aisle_reorder_total_count,aisle_reorder_percentage,aisle_vs_unique_users,aisle,department_avg_add_to_cart_order,department_std_add_to_cart_order,department_orders_total_count,department_reorder_total_count,department_reorder_percentage,department_vs_unique_users,department
0,1,5.801836,1852,1136.0,0.613391,716,276,0,716,0.385475,9.253092,7.845107,234065,128431.0,0.548698,54202,cookies cakes,9.187743,7.692492,2887550,1657973.0,0.574180,174219,snacks
1,2,9.888889,90,12.0,0.133333,78,8,0,78,0.102564,9.996181,7.994790,212092,32321.0,0.152391,76402,spices seasonings,9.593425,7.875241,1875577,650301.0,0.346721,172755,pantry
2,3,6.415162,277,203.0,0.732852,74,36,0,74,0.486486,8.519846,7.818249,249341,131556.0,0.527615,53197,tea,6.976699,6.711172,2690129,1757892.0,0.653460,172795,beverages
3,4,9.507599,329,147.0,0.446809,182,64,0,182,0.351648,9.207741,7.435740,390299,217262.0,0.556655,58749,frozen meals,8.996414,7.393502,2236432,1211890.0,0.541885,163233,frozen
4,5,6.466667,15,9.0,0.600000,6,4,0,6,0.666667,10.297600,8.187047,62510,17542.0,0.280627,32312,marinades meat preparation,9.593425,7.875241,1875577,650301.0,0.346721,172755,pantry


In [46]:
product_level_features.shape

(49677, 24)

In [47]:
product_level_features.dtypes

product_id                            int64
product_avg_add_to_cart_order       float64
product_orders_total_count            int64
product_reorder_total_count         float64
product_reorder_percentage          float64
prod_order_first_time_total_cnt       int64
prod_order_second_time_total_cnt      int64
product_is_organic                    int64
product_vs_unique_users               int64
second_time_purchased_percent       float64
aisle_avg_add_to_cart_order         float64
aisle_std_add_to_cart_order         float64
aisle_orders_total_count              int64
aisle_reorder_total_count           float64
aisle_reorder_percentage            float64
aisle_vs_unique_users                 int64
aisle                                object
department_avg_add_to_cart_order    float64
department_std_add_to_cart_order    float64
department_orders_total_count         int64
department_reorder_total_count      float64
department_reorder_percentage       float64
department_vs_unique_users      

#### Handle Categorical Variables from product_level_features

In [48]:
# Handle categorical variable : department  
department = pd.get_dummies(product_level_features['department'],drop_first=True, prefix = "department" )
product_level_features  = pd.concat([product_level_features,department],axis=1)
product_level_features = product_level_features.drop('department',axis=1)

In [49]:
product_level_features.dtypes

product_id                            int64
product_avg_add_to_cart_order       float64
product_orders_total_count            int64
product_reorder_total_count         float64
product_reorder_percentage          float64
prod_order_first_time_total_cnt       int64
prod_order_second_time_total_cnt      int64
product_is_organic                    int64
product_vs_unique_users               int64
second_time_purchased_percent       float64
aisle_avg_add_to_cart_order         float64
aisle_std_add_to_cart_order         float64
aisle_orders_total_count              int64
aisle_reorder_total_count           float64
aisle_reorder_percentage            float64
aisle_vs_unique_users                 int64
aisle                                object
department_avg_add_to_cart_order    float64
department_std_add_to_cart_order    float64
department_orders_total_count         int64
department_reorder_total_count      float64
department_reorder_percentage       float64
department_vs_unique_users      

In [50]:
# Handle categorical variable : aisle
aisle = pd.get_dummies(product_level_features['aisle'],drop_first=True, prefix = "aisle" )
product_level_features  = pd.concat([product_level_features,aisle],axis=1)
product_level_features = product_level_features.drop('aisle',axis=1)

In [51]:
# Handle categorical variable : eval_set
eval_set = pd.get_dummies(product_purchase_history_df['eval_set'],drop_first=True, prefix = "eval_set" )
product_purchase_history_df  = pd.concat([product_purchase_history_df,eval_set],axis=1)
product_purchase_history_df = product_purchase_history_df.drop('eval_set',axis=1)

In [52]:
#Validate after creating product_level_feature any null record are present or not?
product_level_features.isnull().any().any()

False

In [53]:
#Validate after creating product_purchase_history_df any null record are present or not?
product_purchase_history_df.isnull().any()

order_id                        False
product_id                      False
add_to_cart_order               False
reordered                       False
user_id                         False
order_number                    False
order_dow                       False
order_hour_of_day               False
days_since_prior_order          False
product_name                    False
aisle_id                        False
department_id                   False
user_purchased_product_count    False
dtype: bool

In [54]:
product_purchase_history_df.shape

(32434489, 13)

In [55]:
product_purchase_history_df.dtypes

order_id                          int32
product_id                        int32
add_to_cart_order                 int16
reordered                          int8
user_id                           int32
order_number                       int8
order_dow                          int8
order_hour_of_day                  int8
days_since_prior_order          float16
product_name                     object
aisle_id                          int16
department_id                      int8
user_purchased_product_count      int64
dtype: object

In [56]:
# Checking common variable in between product_purchase_history_df and product_level_features dataset
a = np.intersect1d(product_purchase_history_df.columns, product_level_features.columns)
print (a)


['product_id']


##### Data transformation : User level feature selection

In [57]:
user_level_features = pd.DataFrame(columns=['user_purchased_avg_dow'])

In [58]:
#Feature : user_purchased_avg_dow
user_level_features['user_purchased_avg_dow'] = product_purchase_history_df.groupby('user_id')['order_dow'].mean()

In [59]:
#Feature : user_purchased_std_dow
user_level_features['user_purchased_std_dow'] = product_purchase_history_df.groupby('user_id')['order_dow'].std()

In [60]:
#Feature : user_avg_order_hour_of_day
user_level_features['user_avg_order_hour_of_day'] = product_purchase_history_df.groupby('user_id')['order_hour_of_day'].mean()

In [61]:
#Feature : user_std_order_hour_of_day
user_level_features['user_std_order_hour_of_day'] = product_purchase_history_df.groupby('user_id')['order_hour_of_day'].std()

In [62]:
#Feature : user_avg_days_since_prior_order
user_level_features['user_avg_days_since_prior_order'] = product_purchase_history_df.groupby('user_id')['days_since_prior_order'].mean()

In [63]:
#Feature : user_std_days_since_prior_order
user_level_features['user_std_days_since_prior_order'] = product_purchase_history_df.groupby('user_id')['days_since_prior_order'].std()

In [64]:
#Feature : total_order_by_user
user_level_features['total_order_by_user'] = product_purchase_history_df.groupby('user_id')['order_number'].apply(lambda x: x.nunique())

In [65]:
#Feature : total_product_purchased_by_user
user_level_features['total_product_purchased_by_user'] = product_purchase_history_df.groupby('user_id')['product_id'].count()

In [66]:
#Feature : total_unique_product_purchased_by_user
user_level_features['total_unique_product_purchased_by_user'] = product_purchase_history_df.groupby('user_id')['product_id'].apply(lambda x: x.nunique())

In [67]:
#Feature : total_product_reordered_by_user
user_level_features['total_product_reordered_by_user'] = product_purchase_history_df.groupby('user_id')['reordered'].sum()

In [68]:
#Feature : product_reorder_propotion_by_user
user_level_features['product_reorder_propotion_by_user'] = product_purchase_history_df.groupby('user_id')['reordered'].mean()

In [69]:
user_level_features.dtypes

user_purchased_avg_dow                    float64
user_purchased_std_dow                    float64
user_avg_order_hour_of_day                float64
user_std_order_hour_of_day                float64
user_avg_days_since_prior_order           float16
user_std_days_since_prior_order           float64
total_order_by_user                         int64
total_product_purchased_by_user             int64
total_unique_product_purchased_by_user      int64
total_product_reordered_by_user           float64
product_reorder_propotion_by_user         float64
dtype: object

In [70]:
user_level_features2 = pd.DataFrame(columns=['user_average_order_size'])

In [71]:
#Feature : user_average_order_size
user_level_features2['user_average_order_size'] = product_purchase_history_df.groupby(['user_id', 'order_number'])['reordered'].count()

In [72]:
#Feature : user_reorder_items_in_order
user_level_features2['user_reorder_items_in_order'] = product_purchase_history_df.groupby(['user_id', 'order_number'])['reordered'].mean()

In [73]:
user_level_features2.head(5)

user_average_order_size  user_reorder_items_in_order
user_id order_number                                                      
1       1                                   5                        0.000
        2                                   6                        0.500
        3                                   5                        0.600
        4                                   5                        1.000
        5                                   8                        0.625

In [74]:

user_level_features3 = user_level_features2.groupby('user_id').agg({'user_average_order_size' : 'mean', 
                                   'user_reorder_items_in_order':'mean'})
user_level_features3 = user_level_features3.reset_index()
user_level_features3.head()

,user_id,user_average_order_size,user_reorder_items_in_order
0,1,5.900000,0.705833
1,2,13.928571,0.447961
2,3,7.333333,0.658817
3,4,3.600000,0.028571
4,5,9.250000,0.377778


In [75]:
user_level_features = user_level_features.merge(user_level_features3, on = 'user_id', how = 'left')
user_level_features.head()

,user_id,user_purchased_avg_dow,user_purchased_std_dow,user_avg_order_hour_of_day,user_std_order_hour_of_day,user_avg_days_since_prior_order,user_std_days_since_prior_order,total_order_by_user,total_product_purchased_by_user,total_unique_product_purchased_by_user,total_product_reordered_by_user,product_reorder_propotion_by_user,user_average_order_size,user_reorder_items_in_order
0,1,2.644068,1.256194,10.542373,3.500355,19.140625,9.642681,10,59,18,41.0,0.694915,5.900000,0.705833
1,2,2.005128,0.971222,10.441026,1.649854,15.367188,9.089876,14,195,102,93.0,0.476923,13.928571,0.447961
2,3,1.011364,1.245630,16.352273,1.454599,10.976562,4.799371,12,88,33,55.0,0.625000,7.333333,0.658817
3,4,4.722222,0.826442,13.111111,1.745208,13.500000,8.311933,5,18,17,1.0,0.055556,3.600000,0.028571
4,5,1.621622,1.276961,15.729730,2.588958,12.273438,4.970181,4,37,23,14.0,0.378378,9.250000,0.377778


In [76]:
user_level_features2= user_level_features2.reset_index()

In [77]:
#Feature : last lastest three orders details for a specific  user 
last_three_orders = user_level_features2.groupby('user_id')['order_number'].nlargest(3).reset_index()
last_three_orders.head()

,user_id,level_1,order_number
0,1,9,10
1,1,8,9
2,1,7,8
3,2,23,14
4,2,22,13


In [78]:
last_three_orders = user_level_features2.merge(last_three_orders, on = ['user_id', 'order_number'], how = 'inner')
last_three_orders.head()

,user_id,order_number,user_average_order_size,user_reorder_items_in_order,level_1
0,1,8,6,0.666667,7
1,1,9,6,1.000000,8
2,1,10,9,0.666667,9
3,2,12,19,0.578947,21
4,2,13,9,0.000000,22


In [79]:
last_three_orders['rank'] = last_three_orders.groupby("user_id")["order_number"].rank("dense", ascending=True)

In [80]:
#Feature : based on last lastest three orders details geenerate for a specific  user ( 'user_id','orders_3', 'orders_2', 'orders_1', 'reorder_3', 'reorder_2', 'reorder_1')
last_order_feats = last_three_orders.pivot_table(index = 'user_id', columns = ['rank'], \
                                                 values=['user_average_order_size', 'user_reorder_items_in_order']).\
                                                reset_index(drop = False)
last_order_feats.columns = ['user_id','orders_3', 'orders_2', 'orders_1', 'reorder_3', 'reorder_2', 'reorder_1']
last_order_feats.head()

,user_id,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,6,6,9,0.666667,1.0,0.666667
1,2,19,9,16,0.578947,0.0,0.625000
2,3,6,5,6,0.833333,1.0,1.000000
3,4,7,2,3,0.142857,0.0,0.000000
4,5,9,5,12,0.444444,0.4,0.666667


In [81]:
user_level_features = user_level_features.merge(last_order_feats, on = 'user_id', how = 'left')
user_level_features.head()

,user_id,user_purchased_avg_dow,user_purchased_std_dow,user_avg_order_hour_of_day,user_std_order_hour_of_day,user_avg_days_since_prior_order,user_std_days_since_prior_order,total_order_by_user,total_product_purchased_by_user,total_unique_product_purchased_by_user,total_product_reordered_by_user,product_reorder_propotion_by_user,user_average_order_size,user_reorder_items_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.644068,1.256194,10.542373,3.500355,19.140625,9.642681,10,59,18,41.0,0.694915,5.900000,0.705833,6,6,9,0.666667,1.0,0.666667
1,2,2.005128,0.971222,10.441026,1.649854,15.367188,9.089876,14,195,102,93.0,0.476923,13.928571,0.447961,19,9,16,0.578947,0.0,0.625000
2,3,1.011364,1.245630,16.352273,1.454599,10.976562,4.799371,12,88,33,55.0,0.625000,7.333333,0.658817,6,5,6,0.833333,1.0,1.000000
3,4,4.722222,0.826442,13.111111,1.745208,13.500000,8.311933,5,18,17,1.0,0.055556,3.600000,0.028571,7,2,3,0.142857,0.0,0.000000
4,5,1.621622,1.276961,15.729730,2.588958,12.273438,4.970181,4,37,23,14.0,0.378378,9.250000,0.377778,9,5,12,0.444444,0.4,0.666667


In [82]:
#Validate after creating product_level_feature any null record are present or not?
user_level_features.isnull().any().any()

False

In [83]:
user_level_features.shape

(206209, 20)

##### Data transformation : User vs level feature selection

In [84]:
# Memory reduction for product_purchase_history_df dataset
product_purchase_history_df = reduce_memory(product_purchase_history_df)

Memory usage of properties dataframe is : 1453.8011388778687  MB

___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  1237.2775650024414  MB
This is  85.1063829787234 % of the initial size


In [85]:
user_product_level_features = pd.DataFrame(columns=['total_product_orders_by_user'])

In [86]:
#Feature : total_product_orders_by_user
user_product_level_features['total_product_orders_by_user'] = product_purchase_history_df.groupby(['user_id', 'product_id'])['reordered'].count()

In [87]:
#Feature : total_product_reorders_by_user
user_product_level_features['total_product_reorders_by_user'] = product_purchase_history_df.groupby(['user_id', 'product_id'])['reordered'].sum()

In [88]:
#Feature : avg_add_to_cart_by_user
user_product_level_features['avg_add_to_cart_by_user'] = product_purchase_history_df.groupby(['user_id', 'product_id'])['add_to_cart_order'].mean()

In [89]:
#Feature : avg_days_since_last_bought
user_product_level_features['avg_days_since_last_bought'] = product_purchase_history_df.groupby(['user_id', 'product_id'])['days_since_prior_order'].mean()

In [90]:
#Feature : last_ordered_in
user_product_level_features['last_ordered_in'] = product_purchase_history_df.groupby(['user_id', 'product_id'])['order_number'].max()

In [91]:
last_three_orders.head()

,user_id,order_number,user_average_order_size,user_reorder_items_in_order,level_1,rank
0,1,8,6,0.666667,7,1.0
1,1,9,6,1.000000,8,2.0
2,1,10,9,0.666667,9,3.0
3,2,12,19,0.578947,21,1.0
4,2,13,9,0.000000,22,2.0


In [92]:
last_orders = product_purchase_history_df.merge(last_three_orders, on = ['user_id', 'order_number'], how = 'inner')
last_orders.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_purchased_product_count,user_average_order_size,user_reorder_items_in_order,level_1,rank
0,7,34050,1,0,142903,11,2,14,30.0,Orange Juice,31,7,1,2,0.000000,2231251,2.0
1,7,46802,2,0,142903,11,2,14,30.0,Pineapple Chunks,116,1,1,2,0.000000,2231251,2.0
2,14,20392,1,1,18194,49,3,15,3.0,Hair Bender Whole Bean Coffee,26,7,1,11,0.818182,282882,1.0
3,14,27845,2,1,18194,49,3,15,3.0,Organic Whole Milk,84,16,1,11,0.818182,282882,1.0
4,14,162,3,1,18194,49,3,15,3.0,Organic Mini Homestyle Waffles,52,1,1,11,0.818182,282882,1.0


In [93]:
#Feature : last lastest three orders details for a specific user and specific product
last_orders['rank'] = last_orders.groupby(['user_id', 'product_id'])['order_number'].rank("dense", ascending=True)

In [95]:
user_product_level_features = user_product_level_features.merge(product_purchase_history, on=['user_id', 'product_id'], how = 'left')
user_product_level_features.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,1.0,2.0,3.0
0,1,196,10,9,1.400000,18.296875,10,1.0,1.0,1.0
1,1,10258,9,8,3.333333,19.562500,10,1.0,1.0,1.0
2,1,10326,1,0,5.000000,28.000000,5,NaN,NaN,NaN
3,1,12427,10,9,3.300000,18.296875,10,1.0,1.0,1.0
4,1,13032,3,2,6.333333,21.671875,10,1.0,NaN,NaN


In [96]:
#Validate after creating product_level_feature any null record are present or not?
user_product_level_features.isnull().any().any()

True

In [97]:
# verifying which all feature is having null values.
round(user_product_level_features.isnull().sum(axis=0)/(user_product_level_features.shape[0])*100).sort_values(ascending=False)

3.0                               98.0
2.0                               92.0
1.0                               63.0
last_ordered_in                    0.0
avg_days_since_last_bought         0.0
avg_add_to_cart_by_user            0.0
total_product_reorders_by_user     0.0
total_product_orders_by_user       0.0
product_id                         0.0
user_id                            0.0
dtype: float64

In [106]:
#product_name , aisle_id and department _id will not play any. role for model building . Hence eliminating  from the dataset
user_product_level_features = user_product_level_features.drop(columns=[1.0,2.0,3.0])

In [107]:
# verifying which all feature is having null values.
round(user_product_level_features.isnull().sum(axis=0)/(user_product_level_features.shape[0])*100).sort_values(ascending=False)

last_ordered_in                   0.0
avg_days_since_last_bought        0.0
avg_add_to_cart_by_user           0.0
total_product_reorders_by_user    0.0
total_product_orders_by_user      0.0
product_id                        0.0
user_id                           0.0
dtype: float64

In [108]:
user_product_level_features.shape

(13307953, 7)

In [109]:
product_level_features.to_csv(destination_data+'final_product_level_features.csv')

In [110]:
user_level_features.to_csv(destination_data+'final_user_level_features.csv')

In [111]:
user_product_level_features.to_csv(destination_data+'final_user_product_level_features.csv')